In [9]:
!pip install huggingface_hub
!pip install transformers
!pip install datasets
!pip install shap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 575 kB 5.3 MB/s 


In [21]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer,BertModel,TrainingArguments,Trainer,PreTrainedModel
from transformers import PretrainedConfig
import transformers
from datasets import Dataset

from huggingface_hub import notebook_login

import torch
import torch.nn as nn
from torch.utils.data import RandomSampler, DataLoader,SequentialSampler
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm import notebook
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np

import shap

In [42]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [43]:
%cd drive/MyDrive/'NLP Final Project'

/content/drive/.shortcut-targets-by-id/1idaRkfcIlcNE-cXtRtSk3Qo9QdbhAh4Q/NLP Final Project


In [45]:
final_dataset = pd.read_csv("./final-dataset/final_dataset_new.csv")
final_dataset

,symbol,year,truncated_text,score
0,a,2016,if it meets it will likely react the way the a...,0.884762
1,a,2021,we’ve changed lives for the better and took mo...,0.886514
2,aal,2020,"$jets, $play, &amp; $aal gains [ericsson profi...",0.622026
3,aal,2021,$aal lets gooooooo! $aal letsssss gooooooooooo...,0.685919
4,aap,2016,with all these uber and lyft cars on these str...,0.333698
...,...,...,...,...
837,xray,2021,"as of now, only about ⅓ of align’s customers a...",0.678117
838,xyl,2020,the applied water segment encompasses the uses...,0.826304
839,yum,2021,can nuggets do the same? i don't care.tldr – *...,0.772902
840,zts,2016,$pzza- buy a 75/80 nov16 call spread or sell n...,0.452652


## Defining Word Sets

In [127]:
positive_words = ['ethical', 'renewable', 'democratic']
negative_words = ['pollution', 'corruption', 'harassment']

In [82]:
tokenizer = AutoTokenizer.from_pretrained('yiyanghkust/finbert-esg',model_max_length=256)

In [94]:
classification_model = AutoModelForSequenceClassification.from_pretrained("sunnydigital/finbert-esg-reddit-classification", trust_remote_code=True)
classification_pred = transformers.pipeline("text-classification", model=classification_model, tokenizer=tokenizer, return_all_scores=True, padding=True, truncation=True)

`return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.


In [128]:
classification_explainer = shap.Explainer(classification_pred)
classification_shap_values_positive = classification_explainer(positive_words)
classification_shap_values_negative = classification_explainer(negative_words)

In [129]:
shap.plots.text(classification_shap_values_positive)

In [130]:
shap.plots.text(classification_shap_values_negative)

In [131]:
print(classification_shap_values_positive.values.shape)
print(classification_shap_values_negative.values.shape)

(3,)
(3,)


In [132]:
regression = AutoModelForSequenceClassification.from_pretrained("sunnydigital/finbert-esg-reddit-regression", trust_remote_code=True)
pred_regression = transformers.pipeline("text-classification", model=regression, tokenizer=tokenizer, return_all_scores=True, padding=True, truncation=True)

`return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.


In [133]:
regression_explainer = shap.Explainer(pred_regression)
regression_shap_values_positive = regression_explainer(positive_words)
regression_shap_values_negative = regression_explainer(negative_words)

In [134]:
shap.plots.text(regression_shap_values_positive)

In [135]:
shap.plots.text(regression_shap_values_negative)

In [136]:
print(regression_shap_values_positive.values.shape)
print(regression_shap_values_negative.values.shape)

(3,)
(3,)
